In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_context('paper')
sns.set_style({'axes.axisbelow': True, 
               'axes.edgecolor': '.15',
               'axes.facecolor': 'white',
               'axes.grid': True, 
               'axes.labelcolor': '.15', 
               'figure.facecolor': 'white', 
               'grid.color': '.15',
               'grid.linestyle': ':', 
               'grid.alpha': .5, 
               'image.cmap': 'Greys', 
               'legend.frameon': False, 
               'legend.numpoints': 1, 
               'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'butt', 
               'axes.spines.right': False, 
               'axes.spines.top': False,  
               'text.color': '.15',  
               'xtick.top': False, 
               'ytick.right': False, 
               'xtick.color': '.15',
               'xtick.direction': 'out', 
               'ytick.color': '.15', 
               'ytick.direction': 'out', 
              })


import matplotlib

FONT_SIZE_PT = 5
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.size'] = FONT_SIZE_PT
matplotlib.rcParams['axes.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['axes.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['figure.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['xtick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['ytick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.fontsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.title_fontsize'] = FONT_SIZE_PT

matplotlib.rcParams['xtick.major.size'] = matplotlib.rcParams['ytick.major.size'] = 2
matplotlib.rcParams['xtick.major.width'] = matplotlib.rcParams['ytick.major.width'] = 0.5


matplotlib.rcParams['xtick.minor.size'] = matplotlib.rcParams['ytick.minor.size'] = 1

matplotlib.rcParams['xtick.minor.width'] = matplotlib.rcParams['ytick.minor.width'] = 0.5

matplotlib.rcParams['axes.linewidth'] = 0.5
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['grid.linewidth'] = 0.25
matplotlib.rcParams['patch.linewidth'] = 0.25
matplotlib.rcParams['lines.markeredgewidth'] = 0.25
matplotlib.rcParams['lines.markersize'] = 2

FIVE_MM_IN_INCH = 0.19685
DPI = 600
matplotlib.rcParams['figure.figsize'] = (10 * FIVE_MM_IN_INCH, 9 * FIVE_MM_IN_INCH)
matplotlib.rcParams['savefig.dpi'] = DPI
matplotlib.rcParams['figure.dpi'] = DPI // 4


#http://phyletica.org/matplotlib-fonts/
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# (01) Extracting the dataset.

This notebook handles the first step of analysis - namely the ingestion of the raw dataset and few transformations such as assigning Gene names and readable labels to proteins

## Configuration

Input files (raw dataset)

In [2]:
INPUT_RAW_DATA = {
    'long-linkers-enh': 'data/long_linkers_enhancer_PTMs_research_11_2022.xlsx',
    'long-linkers-prom': 'data/long_linkers_promoter_PTMs_research_11_2022.xlsx',
    'short-linkers': 'data/NPD_short_linkers.xlsx'
}

Output directory

In [3]:
import pathlib
OUTPUT_DIRECTORY = pathlib.Path('outputs') / '01-extracting'

if not OUTPUT_DIRECTORY.is_dir():
    OUTPUT_DIRECTORY.mkdir(parents=True)

## Reading Excel Files

In [4]:
datasets = {
    k: pd.read_excel(v, index_col=0) for k,v in INPUT_RAW_DATA.items()
}

data = pd.concat(datasets.values(), keys=datasets.keys())
data.index.names = ['Dataset', data.index.names[1]]
data.head()

Description  \
Dataset          Accession                                                      
long-linkers-enh P35579     Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 ...   
                 Q13813     Spectrin alpha chain, non-erythrocytic 1 OS=Ho...   
                 P78527     DNA-dependent protein kinase catalytic subunit...   
                 P60709     Actin, cytoplasmic 1 OS=Homo sapiens OX=9606 G...   
                 Q01082     Spectrin beta chain, non-erythrocytic 1 OS=Hom...   

                            Coverage [%]  # Peptides  # PSMs  \
Dataset          Accession                                     
long-linkers-enh P35579               71         170   20013   
                 Q13813               79         205   11500   
                 P78527               64         246   12445   
                 P60709               82          30   10826   
                 Q01082               79         180   10348   

                            # Unique Peptides  # AAs  MW [kDa]  \
Dataset          Accession                                       
long-linkers-enh P35579                   137   1960     226.4   
                 Q13813                   204   2472     284.4   
                 P78527                   246   4128     468.8   
                 P60709                    13    375      41.7   
                 Q01082                   166   2364     274.4   

                            Score Mascot: Mascot  \
Dataset          Accession                         
long-linkers-enh P35579                   639499   
                 Q13813                   391743   
                 P78527                   377897   
                 P60709                   369811   
                 Q01082                   350270   

                            # Peptides (by Search Engine): Mascot  \
Dataset          Accession                                          
long-linkers-enh P35579                                       170   
                 Q13813                                       205   
                 P78527                                       246   
                 P60709                                        30   
                 Q01082                                       180   

                            H3unmod_50bp_repl1  ...  H3K27me3_45bp__3  \
Dataset          Accession                      ...                     
long-linkers-enh P35579           2.012982e+09  ...               NaN   
                 Q13813           7.198720e+08  ...               NaN   
                 P78527           7.189680e+08  ...               NaN   
                 P60709           1.711385e+10  ...               NaN   
                 Q01082           7.444007e+08  ...               NaN   

                            H3K27me3_50bp__3  H3K27me3_55bp__3  \
Dataset          Accession                                       
long-linkers-enh P35579                  NaN               NaN   
                 Q13813                  NaN               NaN   
                 P78527                  NaN               NaN   
                 P60709                  NaN               NaN   
                 Q01082                  NaN               NaN   

                            H3unmod_35bp_3  H3unmod_40bp_3  H3unmod_45bp_3  \
Dataset          Accession                                                   
long-linkers-enh P35579                NaN             NaN             NaN   
                 Q13813                NaN             NaN             NaN   
                 P78527                NaN             NaN             NaN   
                 P60709                NaN             NaN             NaN   
                 Q01082                NaN             NaN             NaN   

                            H3unmod_50bp__3  H3unmod_55bp_3  H3K9me3_35bp_3  \
Dataset          Accession                                                    
long-linkers-enh P35579                 NaN             N

### Parsing gene names

Now we attempt to parse the gene names from the Description Column in the data.

Particularly, the description column contains the following information:

In [5]:
data['Description'].head()

Dataset           Accession
long-linkers-enh  P35579       Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 ...
                  Q13813       Spectrin alpha chain, non-erythrocytic 1 OS=Ho...
                  P78527       DNA-dependent protein kinase catalytic subunit...
                  P60709       Actin, cytoplasmic 1 OS=Homo sapiens OX=9606 G...
                  Q01082       Spectrin beta chain, non-erythrocytic 1 OS=Hom...
Name: Description, dtype: object

E.g. Accession=P35579 has the following description:

Myosin-9 OS=Homo sapiens GN=MYH9 PE=1 SV=4
In this entry we see the segment GN=MYH9 which tells us that the gene name of this protein is MYH9

Note that most proteins have this gene name, encoded in the GN portion of the Description (e.g. GN=MYH9) or `Gene_Symbol` portion (e.g. `Gene_symbol=KRT10` except:

In [6]:
data[~(data['Description'].str.contains('GN') | data['Description'].str.contains('Gene_Symbol'))]['Description'].unique()

array(['Streptavidin OS=Streptomyces avidinii PE=1 SV=1',
       'SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor',
       'SWISS-PROT:P15636 Protease I precursor Lysyl endopeptidase Achromobacter lyticus.',
       'Trypsin - Sus scrofa (Pig).',
       'SWISS-PROT:Q28107 (Bos taurus) Coagulation factor V precursor',
       'Golgin subfamily A member 8-like protein 2 OS=Homo sapiens PE=2 SV=2',
       'TREMBL:Q9UE12 Type I hair keratin 1 - Homo sapiens (Human).',
       'SWISS-PROT:Q14525 Keratin, type I cuticular HA3-II (Hair keratin, type I HA3-II)',
       'SWISS-PROT:O76013 Keratin, type I cuticular HA6 (Hair keratin, type I HA6)',
       'SWISS-PROT:P78386 Keratin, type II cuticular Hb5 (Hair keratin, type II Hb5) - Homo sapiens (Human).',
       'SWISS-PROT:Q9NSB2 Keratin, type II cuticular Hb4 (Hair keratin, type II Hb4) - Homo sapiens (Human).',
       'TREMBL:Q8IUT8 Type I hair keratin 4 - Homo sapiens (Human).',
       'SWISS-PROT:Q3SZR3 (Bos taurus) Alpha-1-acid

The above proteins are not particularly important for us and therefore it should be safe to parse the gene names from this columns, as long as we have a fallback for the exceptions above

In [7]:
import re
def parse_gn(description):
    """
    Parses the gene name from `GN=ABCD`, or `Gene_Symbol=ABCD` like string in the description
    """
    
    match = re.match('.*(GN|Gene_Symbol)=(?P<gene_name>.*?)(\s+|$)', description)
    if match:
        return match.group('gene_name').strip()
    else:
        return None
data['Gene'] = data['Description'].map(parse_gn)

In [8]:
data[data['Description'].str.contains('GN')]['Description'].iloc[0]

'Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 SV=4'

In [9]:
data[data['Description'].str.contains('Gene_Symbol')]['Description'].iloc[0]

'SWISS-PROT:P35527 Tax_Id=9606 Gene_Symbol=KRT9 Keratin, type I cytoskeletal 9'

In [10]:
parse_gn('Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 SV=4')

'MYH9'

In [11]:
parse_gn('SWISS-PROT:P35527 Tax_Id=9606 Gene_Symbol=KRT9 Keratin, type I cytoskeletal 9')

'KRT9'

As expected, the parsing failed for only one gene:

In [12]:
data[data['Gene'].isnull()]['Description'].unique()

array(['Streptavidin OS=Streptomyces avidinii PE=1 SV=1',
       'SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor',
       'SWISS-PROT:P15636 Protease I precursor Lysyl endopeptidase Achromobacter lyticus.',
       'Trypsin - Sus scrofa (Pig).',
       'SWISS-PROT:Q28107 (Bos taurus) Coagulation factor V precursor',
       'Golgin subfamily A member 8-like protein 2 OS=Homo sapiens PE=2 SV=2',
       'TREMBL:Q9UE12 Type I hair keratin 1 - Homo sapiens (Human).',
       'SWISS-PROT:Q14525 Keratin, type I cuticular HA3-II (Hair keratin, type I HA3-II)',
       'SWISS-PROT:O76013 Keratin, type I cuticular HA6 (Hair keratin, type I HA6)',
       'SWISS-PROT:P78386 Keratin, type II cuticular Hb5 (Hair keratin, type II Hb5) - Homo sapiens (Human).',
       'SWISS-PROT:Q9NSB2 Keratin, type II cuticular Hb4 (Hair keratin, type II Hb4) - Homo sapiens (Human).',
       'TREMBL:Q8IUT8 Type I hair keratin 4 - Homo sapiens (Human).',
       'SWISS-PROT:Q3SZR3 (Bos taurus) Alpha-1-acid

To account for the exceptions, we create another column Label, which will be set to Gene, when the Gene is present and to Accesion, when it is not.

In [13]:
data['Label'] = data['Gene'].copy() # set to Gene
# Where gene is null; set to the Accession (i.e. index)
data.loc[data['Gene'].isnull(), 'Label'] = [ix[1] for ix in data.loc[data['Gene'].isnull()].index]

We now have the scenario that Label column is never null (as we would expect):

In [14]:
assert not data['Label'].isnull().any()

But unfortunately, some labels may be duplicated (as some gene names are duplicated):

In [15]:
for _dataset, subdf in data.groupby(level='Dataset'):
    
    # Find duplicates:
    subdf_duplicates = subdf[subdf['Label'].duplicated(keep=False)]
    has_duplicates = not subdf_duplicates.empty
    print('Dataset {} has duplicated labels: {}'.format(_dataset, has_duplicates))
    
    if not has_duplicates:
        continue
    
    # Resolve duplicates by adding suffix (1), (2), ... etc.
    for label, duplicate_indices in subdf_duplicates.groupby('Label'):
        duplicate_indices = duplicate_indices.sort_index()
        for i, (ix, row) in enumerate(duplicate_indices.iterrows(), start=1):
            old_label = row['Label']
            new_label = '{} ({})'.format(row['Label'], i)
            
            print(f'Renaming {old_label} ({ix[1]}) to {new_label}') 
            data.loc[ix, 'Label'] = new_label
            
    
    
    
    break
#     for (dataset, label), subduplicates in _duplicates.groupby(['Dataset', 'Label']):
#     subduplicates.sort_index()
#     for ix in enumerate(subduplicates.index, start=1):
#         subdf.loc[ix] = f'{label} ({i})'
    

Dataset long-linkers-enh has duplicated labels: False
Dataset long-linkers-prom has duplicated labels: False
Dataset short-linkers has duplicated labels: True
Renaming CALU (O43852) to CALU (1)
Renaming CALU (O43852-4) to CALU (2)
Renaming CENPV (Q7Z7K6) to CENPV (1)
Renaming CENPV (Q7Z7K6-3) to CENPV (2)
Renaming RBM14 (Q96PK6) to RBM14 (1)
Renaming RBM14 (Q96PK6-5) to RBM14 (2)
Renaming SORBS2 (O94875-11) to SORBS2 (1)
Renaming SORBS2 (O94875-8) to SORBS2 (2)
Renaming SPTAN1 (Q13813-2) to SPTAN1 (1)
Renaming SPTAN1 (Q13813-3) to SPTAN1 (2)
Renaming TPM1 (P09493-3) to TPM1 (1)
Renaming TPM1 (P09493-5) to TPM1 (2)
Renaming TPM4 (P67936) to TPM4 (1)
Renaming TPM4 (P67936-2) to TPM4 (2)


Check that no labels are duplicated any more:

In [16]:
for _dataset, subdf in data.groupby(level='Dataset'):
    
    # Find duplicates:
    subdf_duplicates = subdf[subdf['Label'].duplicated(keep=False)]
    has_duplicates = not subdf_duplicates.empty
    print('Dataset {} has duplicated labels: {}'.format(_dataset, has_duplicates))
    
    assert not has_duplicates

Dataset long-linkers-enh has duplicated labels: False
Dataset long-linkers-prom has duplicated labels: False
Dataset short-linkers has duplicated labels: False


And therefore we can now set a natural index to our data, i.e. the Label column:

In [17]:
data = data.reset_index().set_index(['Dataset', 'Label'])
data

Accession  \
Dataset          Label              
long-linkers-enh MYH9      P35579   
                 SPTAN1    Q13813   
                 PRKDC     P78527   
                 ACTB      P60709   
                 SPTBN1    Q01082   
...                           ...   
short-linkers    PPIH      O43447   
                 REST      Q13127   
                 ZMYM2     Q9UBW7   
                 RYR2      Q92736   
                 NES       P48681   

                                                               Description  \
Dataset          Label                                                       
long-linkers-enh MYH9    Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 ...   
                 SPTAN1  Spectrin alpha chain, non-erythrocytic 1 OS=Ho...   
                 PRKDC   DNA-dependent protein kinase catalytic subunit...   
                 ACTB    Actin, cytoplasmic 1 OS=Homo sapiens OX=9606 G...   
                 SPTBN1  Spectrin beta chain, non-erythrocytic 1 OS=Hom...   
...                                                                    ...   
short-linkers    PPIH    Peptidyl-prolyl cis-trans isomerase H OS=Homo ...   
                 REST    RE1-silencing transcription factor OS=Homo sap...   
                 ZMYM2   Zinc finger MYM-type protein 2 OS=Homo sapiens...   
                 RYR2    Ryanodine receptor 2 OS=Homo sapiens GN=RYR2 P...   
                 NES               Nestin OS=Homo sapiens GN=NES PE=1 SV=2   

                         Coverage [%]  # Peptides  # PSMs  # Unique Peptides  \
Dataset          Label                                                         
long-linkers-enh MYH9              71         170   20013                137   
                 SPTAN1            79         205   11500                204   
                 PRKDC             64         246   12445                246   
                 ACTB              82          30   10826                 13   
                 SPTBN1            79         180   10348                166   
...                               ...         ...     ...                ...   
short-linkers    PPIH              11           2       2                  2   
                 REST               3           2       2                  2   
                 ZMYM2              3           2       4                  2   
                 RYR2               0           2       2                  2   
                 NES                1           2      34                  2   

                         # AAs  MW [kDa]  Score Mascot: Mascot  \
Dataset          Label                                           
long-linkers-enh MYH9     1960     226.4                639499   
                 SPTAN1   2472     284.4                391743   
                 PRKDC    4128     468.8                377897   
                 ACTB      375      41.7                369811   
                 SPTBN1   2364     274.4                350270   
...                        ...       ...                   ...   
short-linkers    PPIH      177      19.2                    20   
                 REST     1097     121.8                    19   
                 ZMYM2    1377     154.8                    18   
                 RYR2     4967     564.2                    15   
                 NES      1621     177.3                     0   

                         # Peptides (by Search Engine): Mascot  ...  \
Dataset          Label                                          ...   
long-linkers-enh MYH9                                      170  ...   
                 SPTAN1                                    205  ...   
                 PRKDC                                     246  ...   
                 ACTB                                       30  ...   
                 SPTBN1                                    180  ...   
...                                                        ...  ...   
short-linkers    PPIH                                      

Splitting numeric data and metadata

At this point it makes sense to split the data into numeric columns and the remaining metadata.
This makes more sense to do for each dataset separately

In [18]:
NUMERIC_COLUMNS = {
     'long-linkers-enh': [
       'H3unmod_50bp_repl1',
     'H3K27ac_50bp_repl1',
     'H3K27ac_200bp_scr_repl1',
     'H3K27ac_200bp_SV40enh_repl1',
     'H3K4me1K27ac_50bp_repl1',
     'H3K4me1K27ac_200bp_scr_repl1',
     'H3K4me1K27ac_200bp_SV40enh_repl1',
     'H3unmod_200bp_scr_repl1',
     'H3unmod_200bp_SV40enh_repl1',
     'H3K4me1_50bp_repl1',
     'H3K4me1_200bp_scr_repl1',
     'H3K4me1_200bp_SV40enh_repl1',
     'H3unmod_50bp_repl2',
     'H3K27ac_50bp_repl2',
     'H3K27ac_200bp_scr_repl2',
     'H3K27ac_200bp_SV40enh_repl2',
     'H3K4me1K27ac_50bp_repl2',
     'H3K4me1K27ac_200bp_scr_repl2',
     'H3K4me1K27ac_200bp_SV40enh_repl2',
     'H3unmod_200bp_scr_repl2',
     'H3unmod_200bp_SV40enh_repl2',
     'H3K4me1_50bp_repl2',
     'H3K4me1_200bp_scr_repl2',
     'H3K4me1_200bp_SV40enh_repl2',
     'H3unmod_50bp_repl3',
     'H3K27ac_50bp_repl3',
     'H3K27ac_200bp_scr_repl3',
     'H3K27ac_200bp_SV40enh_repl3',
     'H3K4me1K27ac_50bp_repl3',
     'H3K4me1K27ac_200bp_scr_repl3',
     'H3K4me1K27ac_200bp_SV40enh_repl3',
     'H3unmod_200bp_scr_repl3',
     'H3unmod_200bp_SV40enh_repl3',
     'H3K4me1_50bp_repl3',
     'H3K4me1_200bp_scr_repl3',
     'H3K4me1_200bp_SV40enh_repl3',
    ],
    'long-linkers-prom': [
         'Promoter_PTMs_50bp_repl1',
         'Promoter_PTMs_200bp_scr_repl1',
         'Promoter_PTMs_200bp_SV40prom_repl1',
         'unmodH3_unmodH4_50bp_repl1',
         'unmodH3_unmodH5_200bp_scr_repl1',
         'unmodH3_unmodH6_200bp_SV40prom_repl1',
         'Promoter_PTMs_50bp_repl2',
         'Promoter_PTMs_200bp_scr_repl2',
         'Promoter_PTMs_200bp_SV40prom_repl2',
         'unmodH3_unmodH4_50bp_repl2',
         'unmodH3_unmodH5_200bp_scr_repl2',
         'unmodH3_unmodH6_200bp_SV40prom_repl2',
         'Promoter_PTMs_50bp_repl3',
         'Promoter_PTMs_200bp_scr_repl3',
         'Promoter_PTMs_200bp_SV40prom_repl3',
         'unmodH3_unmodH4_50bp_repl3',
         'unmodH3_unmodH5_200bp_scr_repl3',
         'unmodH3_unmodH6_200bp_SV40prom_repl3',
    ],
    'short-linkers': [
       'H3K9me3_45bp__1', 'H3K9me3_50bp__1', 'H3K9me3_55bp__1',
       'H3K27me3_35bp__1', 'H3K27me3_40bp__1', 'H3K27me3_45bp__1',
       'H3K27me3_50bp__1', 'H3K27me3_55bp__1', 'H3unmod_35bp_1',
       'H3unmod_40bp_1', 'H3unmod_45bp_1', 'H3unmod_50bp__1', 'H3unmod_55bp_1',
       'H3K9me3_35bp_1', 'H3K9me3_40bp_1', 'H3K9me3_45bp__2',
       'H3K9me3_50bp__2', 'H3K9me3_55bp__2', 'H3K27me3_35bp__2',
       'H3K27me3_40bp__2', 'H3K27me3_45bp__2', 'H3K27me3_50bp__2',
       'H3K27me3_55bp__2', 'H3unmod_35bp_2', 'H3unmod_40bp_2',
       'H3unmod_45bp_2', 'H3unmod_50bp__2', 'H3unmod_55bp_2', 'H3K9me3_35bp_2',
       'H3K9me3_40bp_2', 'H3K9me3_45bp__3', 'H3K9me3_50bp__3',
       'H3K9me3_55bp__3', 'H3K27me3_35bp__3', 'H3K27me3_40bp__3',
       'H3K27me3_45bp__3', 'H3K27me3_50bp__3', 'H3K27me3_55bp__3',
       'H3unmod_35bp_3', 'H3unmod_40bp_3', 'H3unmod_45bp_3', 'H3unmod_50bp__3',
       'H3unmod_55bp_3', 'H3K9me3_35bp_3', 'H3K9me3_40bp_3'
    ],
}

In [19]:
RENAMES = {
    # Fix some typos in column names
    'long-linkers-prom': {
      
        'unmodH3_unmodH5_200bp_scr_repl1': 'unmodH3_unmodH4_200bp_scr_repl1',
        'unmodH3_unmodH6_200bp_SV40prom_repl1': 'unmodH3_unmodH4_200bp_SV40prom_repl1',
        'unmodH3_unmodH5_200bp_scr_repl2': 'unmodH3_unmodH4_200bp_scr_repl2',
        'unmodH3_unmodH6_200bp_SV40prom_repl2': 'unmodH3_unmodH4_200bp_SV40prom_repl2',
        'unmodH3_unmodH5_200bp_scr_repl3': 'unmodH3_unmodH4_200bp_scr_repl3',
        'unmodH3_unmodH6_200bp_SV40prom_repl3': 'unmodH3_unmodH4_200bp_SV40prom_repl3',
        
    }
}

In [20]:
data_numeric = {}
data_metadata = {}

for _dataset in INPUT_RAW_DATA:
    print(_dataset)
    subdf = data.loc[_dataset].dropna(axis=1, how='all')
    renames = RENAMES.get(_dataset, {})
    
    subdf_numeric = subdf[NUMERIC_COLUMNS[_dataset]].rename(columns=renames)
    subdf_metadata = subdf[subdf.columns.difference(subdf_numeric.columns)]
    
    data_numeric[_dataset] = subdf_numeric
    data_metadata[_dataset] = subdf_metadata
    

long-linkers-enh
long-linkers-prom
short-linkers


In [21]:
data_numeric['long-linkers-enh'].head()

,H3unmod_50bp_repl1,H3K27ac_50bp_repl1,H3K27ac_200bp_scr_repl1,H3K27ac_200bp_SV40enh_repl1,H3K4me1K27ac_50bp_repl1,H3K4me1K27ac_200bp_scr_repl1,H3K4me1K27ac_200bp_SV40enh_repl1,H3unmod_200bp_scr_repl1,H3unmod_200bp_SV40enh_repl1,H3K4me1_50bp_repl1,...,H3K27ac_200bp_scr_repl3,H3K27ac_200bp_SV40enh_repl3,H3K4me1K27ac_50bp_repl3,H3K4me1K27ac_200bp_scr_repl3,H3K4me1K27ac_200bp_SV40enh_repl3,H3unmod_200bp_scr_repl3,H3unmod_200bp_SV40enh_repl3,H3K4me1_50bp_repl3,H3K4me1_200bp_scr_repl3,H3K4me1_200bp_SV40enh_repl3
Label,,,,,,,,,,,,,,,,,,,,,
MYH9,2.012982e+09,2.435018e+09,2.687300e+09,2.974732e+09,2.815155e+09,2.664790e+09,2.206769e+09,2.937641e+09,1.541227e+09,2.223824e+09,...,1.625335e+09,1.808056e+09,1.905528e+09,2.050247e+09,1.285533e+09,2.407647e+09,1.993102e+09,1.377992e+09,2.111906e+09,2.361874e+09
SPTAN1,7.198720e+08,9.610952e+08,1.060747e+09,1.057862e+09,1.056938e+09,9.690962e+08,7.797758e+08,1.022564e+09,7.093685e+08,9.102410e+08,...,6.559300e+08,7.782214e+08,8.114757e+08,8.113995e+08,5.240397e+08,9.736662e+08,8.067823e+08,7.037297e+08,8.212826e+08,9.360189e+08
PRKDC,7.189680e+08,8.020305e+08,1.040543e+09,1.030530e+09,8.354429e+08,8.790567e+08,7.307067e+08,8.629810e+08,8.357701e+08,1.062908e+09,...,8.284987e+08,7.411441e+08,9.397634e+08,7.519692e+08,5.829893e+08,1.002489e+09,7.611802e+08,6.041735e+08,8.331257e+08,8.430505e+08
ACTB,1.711385e+10,2.030884e+10,2.143226e+10,1.960820e+10,2.060654e+10,1.932797e+10,1.743471e+10,2.114668e+10,1.532196e+10,2.179764e+10,...,1.425466e+10,1.813711e+10,1.933376e+10,1.845758e+10,1.222580e+10,2.127657e+10,1.772308e+10,1.732659e+10,1.504822e+10,2.033082e+10
SPTBN1,7.444007e+08,1.283232e+09,1.334341e+09,1.233183e+09,1.224636e+09,1.386340e+09,1.237537e+09,7.938937e+08,5.858394e+08,8.826573e+08,...,1.026505e+09,1.317820e+09,1.265981e+09,2.981148e+09,9.649839e+08,9.629035e+08,8.020600e+08,6.267472e+08,4.881266e+09,8.424549e+08


In [22]:
data_numeric['long-linkers-prom'].head()

,Promoter_PTMs_50bp_repl1,Promoter_PTMs_200bp_scr_repl1,Promoter_PTMs_200bp_SV40prom_repl1,unmodH3_unmodH4_50bp_repl1,unmodH3_unmodH4_200bp_scr_repl1,unmodH3_unmodH4_200bp_SV40prom_repl1,Promoter_PTMs_50bp_repl2,Promoter_PTMs_200bp_scr_repl2,Promoter_PTMs_200bp_SV40prom_repl2,unmodH3_unmodH4_50bp_repl2,unmodH3_unmodH4_200bp_scr_repl2,unmodH3_unmodH4_200bp_SV40prom_repl2,Promoter_PTMs_50bp_repl3,Promoter_PTMs_200bp_scr_repl3,Promoter_PTMs_200bp_SV40prom_repl3,unmodH3_unmodH4_50bp_repl3,unmodH3_unmodH4_200bp_scr_repl3,unmodH3_unmodH4_200bp_SV40prom_repl3
Label,,,,,,,,,,,,,,,,,,
MYH9,1.281909e+09,2.134530e+09,2.497327e+09,2.281093e+09,2.617519e+09,2.035759e+09,2.860328e+09,1.788147e+09,1.796727e+09,2.444018e+09,3.461079e+09,1.853011e+09,2.199156e+09,1.279729e+09,2.383846e+09,2.562314e+09,2.035037e+09,1.776609e+09
SPTAN1,5.248911e+08,6.587411e+08,8.357382e+08,8.585309e+08,8.809251e+08,7.448489e+08,9.913368e+08,5.434298e+08,5.535093e+08,9.210341e+08,1.152325e+09,7.271738e+08,8.134845e+08,6.484779e+08,6.929144e+08,9.034719e+08,8.427988e+08,7.925378e+08
PRKDC,7.270430e+08,5.506399e+08,7.164303e+08,8.096481e+08,8.800427e+08,6.159315e+08,6.545611e+08,3.778371e+08,4.057029e+08,8.018767e+08,8.223814e+08,4.933144e+08,5.859108e+08,1.037794e+09,6.985676e+08,6.756472e+08,6.079810e+08,6.181795e+08
SPTBN1,7.211787e+08,1.222698e+09,1.586194e+09,8.392360e+08,9.849796e+08,5.236452e+08,1.537787e+09,1.079691e+09,1.061178e+09,7.484772e+08,1.188865e+09,5.496364e+08,1.151616e+09,1.037424e+09,1.468142e+09,7.626874e+08,7.865917e+08,8.272016e+08
ACTB,1.269655e+10,1.720074e+10,1.922848e+10,1.889111e+10,2.085801e+10,1.792695e+10,2.213478e+10,1.323739e+10,1.255952e+10,2.200936e+10,2.567843e+10,1.487529e+10,1.953446e+10,1.416961e+10,1.765019e+10,2.113211e+10,1.921924e+10,1.820500e+10


In [23]:
data_numeric['short-linkers'].head()

,H3K9me3_45bp__1,H3K9me3_50bp__1,H3K9me3_55bp__1,H3K27me3_35bp__1,H3K27me3_40bp__1,H3K27me3_45bp__1,H3K27me3_50bp__1,H3K27me3_55bp__1,H3unmod_35bp_1,H3unmod_40bp_1,...,H3K27me3_45bp__3,H3K27me3_50bp__3,H3K27me3_55bp__3,H3unmod_35bp_3,H3unmod_40bp_3,H3unmod_45bp_3,H3unmod_50bp__3,H3unmod_55bp_3,H3K9me3_35bp_3,H3K9me3_40bp_3
Label,,,,,,,,,,,,,,,,,,,,,
MYH9,6.819964e+09,3.963794e+09,2.897327e+09,2.612789e+09,3.547799e+09,5.158553e+09,4.680400e+09,3.737984e+09,5.536170e+09,4.129462e+09,...,4.917671e+09,3.194571e+09,2.376342e+09,3.601253e+09,3.044944e+09,2.370098e+09,2.235995e+09,2.117592e+09,2.953362e+09,3.526135e+09
CONT_P22629,1.678608e+11,9.968195e+10,8.049941e+10,8.428746e+10,1.168859e+11,1.351942e+11,1.204745e+11,1.083326e+11,1.652194e+11,1.718156e+11,...,1.543118e+11,1.635709e+11,1.490961e+11,1.250698e+11,1.315705e+11,1.476743e+11,1.501223e+11,1.434065e+11,1.102874e+11,1.734283e+11
SPTAN1 (1),4.115371e+08,3.049329e+08,3.010968e+08,2.392642e+08,3.259265e+08,2.979199e+08,3.541832e+08,3.175957e+08,3.776293e+08,4.368028e+08,...,3.154580e+08,2.743185e+08,2.136172e+08,2.431761e+08,2.975633e+08,2.085097e+08,2.214926e+08,2.166143e+08,2.752575e+08,2.904916e+08
SPTAN1 (2),9.686431e+07,6.974094e+07,1.056995e+08,2.127134e+07,3.049779e+07,3.668813e+07,2.716616e+07,9.795320e+07,3.772729e+07,2.814881e+07,...,6.329383e+07,1.368255e+07,3.001479e+07,2.126239e+07,1.273656e+07,3.996528e+07,3.643755e+07,2.781690e+07,2.126948e+07,2.714264e+07
SPTBN1,8.423170e+08,4.273369e+08,3.779590e+08,3.116994e+08,4.303684e+08,5.639140e+08,5.239536e+08,4.354570e+08,6.862634e+08,5.744966e+08,...,5.725942e+08,3.604934e+08,2.890564e+08,4.099113e+08,3.767596e+08,3.096504e+08,3.130653e+08,2.897160e+08,3.523080e+08,4.485469e+08


In [24]:
data_metadata['long-linkers-enh'].head()

,# AAs,# PSMs,# Peptides,# Peptides (by Search Engine): Mascot,# Unique Peptides,Accession,Coverage [%],Description,Gene,MW [kDa],Score Mascot: Mascot
Label,,,,,,,,,,,
MYH9,1960,20013,170,170,137,P35579,71,Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 ...,MYH9,226.4,639499
SPTAN1,2472,11500,205,205,204,Q13813,79,"Spectrin alpha chain, non-erythrocytic 1 OS=Ho...",SPTAN1,284.4,391743
PRKDC,4128,12445,246,246,246,P78527,64,DNA-dependent protein kinase catalytic subunit...,PRKDC,468.8,377897
ACTB,375,10826,30,30,13,P60709,82,"Actin, cytoplasmic 1 OS=Homo sapiens OX=9606 G...",ACTB,41.7,369811
SPTBN1,2364,10348,180,180,166,Q01082,79,"Spectrin beta chain, non-erythrocytic 1 OS=Hom...",SPTBN1,274.4,350270


In [25]:
data_metadata['long-linkers-prom'].head()

,# AAs,# PSMs,# Peptides,# Peptides (by Search Engine): Mascot,# Unique Peptides,Accession,Coverage [%],Description,Gene,MW [kDa],Score Mascot: Mascot,unmodH3_unmodH5_200bp_scr_repl1,unmodH3_unmodH5_200bp_scr_repl2,unmodH3_unmodH5_200bp_scr_repl3,unmodH3_unmodH6_200bp_SV40prom_repl1,unmodH3_unmodH6_200bp_SV40prom_repl2,unmodH3_unmodH6_200bp_SV40prom_repl3
Label,,,,,,,,,,,,,,,,,
MYH9,1960,9223,158,158,137,P35579,69,Myosin-9 OS=Homo sapiens OX=9606 GN=MYH9 PE=1 ...,MYH9,226.4,297789,2.617519e+09,3.461079e+09,2.035037e+09,2.035759e+09,1.853011e+09,1.776609e+09
SPTAN1,2472,5575,203,203,201,Q13813,77,"Spectrin alpha chain, non-erythrocytic 1 OS=Ho...",SPTAN1,284.4,189967,8.809251e+08,1.152325e+09,8.427988e+08,7.448489e+08,7.271738e+08,7.925378e+08
PRKDC,4128,5822,228,228,228,P78527,55,DNA-dependent protein kinase catalytic subunit...,PRKDC,468.8,175811,8.800427e+08,8.223814e+08,6.079810e+08,6.159315e+08,4.933144e+08,6.181795e+08
SPTBN1,2364,4865,165,165,152,Q01082,75,"Spectrin beta chain, non-erythrocytic 1 OS=Hom...",SPTBN1,274.4,167574,9.849796e+08,1.188865e+09,7.865917e+08,5.236452e+08,5.496364e+08,8.272016e+08
ACTB,375,4951,29,29,11,P60709,82,"Actin, cytoplasmic 1 OS=Homo sapiens OX=9606 G...",ACTB,41.7,164631,2.085801e+10,2.567843e+10,1.921924e+10,1.792695e+10,1.487529e+10,1.820500e+10


In [26]:
data_metadata['short-linkers'].head()

,# AAs,# PSMs,# Peptides,# Peptides (by Search Engine): Mascot,# Unique Peptides,Accession,Coverage [%],Description,Gene,MW [kDa],Score Mascot: Mascot,calc. pI
Label,,,,,,,,,,,,
MYH9,1960,35190,178,178,142,P35579,71,Myosin-9 OS=Homo sapiens GN=MYH9 PE=1 SV=4,MYH9,226.4,1244592,5.60
CONT_P22629,183,13859,9,9,9,CONT_P22629,43,Streptavidin OS=Streptomyces avidinii PE=1 SV=1,None,18.8,639027,8.35
SPTAN1 (1),2477,14713,177,177,4,Q13813-2,74,"Isoform 2 of Spectrin alpha chain, non-erythro...",SPTAN1,284.9,543452,5.35
SPTAN1 (2),2452,14454,175,175,2,Q13813-3,74,"Isoform 3 of Spectrin alpha chain, non-erythro...",SPTAN1,282.1,534079,5.34
SPTBN1,2364,12324,148,148,16,Q01082,73,"Spectrin beta chain, non-erythrocytic 1 OS=Hom...",SPTBN1,274.4,455043,5.57


And this is pretty much everything that we needed to do in this notebook, so let's just save the outputs

In [27]:
for k, df in data_numeric.items():
    df.to_csv(OUTPUT_DIRECTORY / f'data_numeric.{k}.csv')
    
for k, df in data_metadata.items():
    df.to_csv(OUTPUT_DIRECTORY / f'data_metadata.{k}.csv')